<a href="https://colab.research.google.com/github/open-mmlab/mmaction2/blob/master/demo/mmaction2_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMAction2 Tutorial

Welcome to MMAction2! This is the official colab tutorial for using MMAction2. In this tutorial, you will learn
- Perform inference with a MMAction2 recognizer.
- Train a new recognizer with a new dataset.

Let's start!

## Install MMAction2

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==latest+torch1.5.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2

!pip install -e .

# Install some optional requirements
!pip install -r requirements/optional.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Requirement already up-to-date: torch==1.5.1+cu101 in /usr/local/lib/python3.6/dist-packages (1.5.1+cu101)
Requirement already up-to-date: torchvision==0.6.1+cu101 in /usr/local/lib/python3.6/dist-packages (0.6.1+cu101)
Looking in links: https://download.openmmlab.com/mmcv/dist/index.html
  Using cached https://download.openmmlab.com/mmcv/dist/latest/torch1.5.0/cu101/mmcv_full-latest%2Btorch1.5.0%2Bcu101-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: mmcv-full 1.2.0
    Uninstalling mmcv-full-1.2.0:
      Successfully uninstalled mmcv-full-1.2.0
Cloning into 'mmaction2'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 7596 (delta 0), reused 1 (delta 0), pack-reused 7588
Receiving objects: 100% (7596/7596), 33.53 MiB | 37.44 MiB/s, done.
Resolving deltas: 100% (5329/5329), done.
/content/mmaction2
Obtaining

In [4]:
#mmaction2 dir
%cd "/home/administrator/Z/Algorithms/mmaction2/"

/home/administrator/Z/Algorithms/mmaction2


In [5]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.8.1 True
0.14.0


ModuleNotFoundError: No module named 'mmcv._ext'

## Perform inference with a MMAction2 recognizer
MMAction2 already provides high level APIs to do inference and training.

In [6]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

--2021-05-14 10:32:04--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 8.208.41.36
Connecting to download.openmmlab.com (download.openmmlab.com)|8.208.41.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97579339 (93M) [application/octet-stream]
Saving to: ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’

checkpoints/tsn_r50 100%[===================>]  93,06M  9,44MB/s    in 10s     

utime(checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth): Operation not permitted
2021-05-14 10:32:15 (9,09 MB/s) - ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’ saved [97579339/97579339]



In [7]:
from mmaction.apis import inference_recognizer, init_recognizer

# Choose to use a config and initialize the recognizer
config = 'configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py'
# Setup a checkpoint file to load
checkpoint = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0')

Use load_from_local loader


In [8]:
# Use the recognizer to do inference
video = 'demo/demo.mp4'
label = 'demo/label_map_k400.txt'
results = inference_recognizer(model, video, label)

In [9]:
# Let's show the results
for result in results:
    print(f'{result[0]}: ', result[1])

arm wrestling:  29.616438
rock scissors paper:  10.75484
shaking hands:  9.908401
clapping:  9.189913
massaging feet:  8.3053055


## Train a recognizer on customized dataset

To train a new recognizer, there are usually three things to do:
1. Support a new dataset
2. Modify the config
3. Train a new recognizer

### Support a new dataset

In this tutorial, we give an example to convert the data into the format of existing datasets. Other methods and more advanced usages can be found in the [doc](/docs/tutorials/new_dataset.md)

Firstly, let's download a tiny dataset obtained from [Kinetics-400](https://deepmind.com/research/open-source/open-source-datasets/kinetics/). We select 30 videos with their labels as train dataset and 10 videos with their labels as test dataset.

In [1]:
#mmaction2.data dir
%cd "/home/administrator/Z/Algorithms/mmaction2/data"

/home/administrator/Z/Algorithms/mmaction2/data


In [2]:
# download, decompress the data
!sudo -S rm kinetics400_tiny.zip* < /home/administrator/Desktop/remote.txt #< /home/administrator/Desktop/remote.txt
!rm -rf kinetics400_tiny
!wget https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
!unzip kinetics400_tiny.zip > /dev/null

rm: cannot remove 'kinetics400_tiny.zip*': No such file or directory
--2021-05-14 10:46:08--  https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 8.208.41.36
Connecting to download.openmmlab.com (download.openmmlab.com)|8.208.41.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18308682 (17M) [application/zip]
Saving to: ‘kinetics400_tiny.zip’

kinetics400_tiny.zi 100%[===================>]  17,46M  5,48MB/s    in 3,4s    

utime(kinetics400_tiny.zip): Operation not permitted
2021-05-14 10:46:13 (5,12 MB/s) - ‘kinetics400_tiny.zip’ saved [18308682/18308682]

chmod (directory attributes) error: Operation not permitted
fchmod (file attributes) error: Operation not permitted
 (warning) cannot set modif./access times
          Operation not permittedfchmod (file attributes) error: Operation not permitted
 (warning) cannot set modif./access times
          Operation not permittedchmod (directory

In [5]:
# Check the directory structure of the tiny data

# Install tree first
!sudo -S apt-get -q install tree < /home/administrator/Desktop/remote.txt
!tree kinetics400_tiny

[sudo] password for administrator: Reading package lists...
Building dependency tree...
Reading state information...
The following packages were automatically installed and are no longer required:
  linux-hwe-5.4-headers-5.4.0-42 linux-hwe-5.4-headers-5.4.0-66
  linux-hwe-5.4-headers-5.4.0-67 linux-hwe-5.4-headers-5.4.0-70
  linux-hwe-5.4-headers-5.4.0-71
Use 'sudo apt autoremove' to remove them.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 120 not upgraded.
Need to get 40,7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://pt.archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40,7 kB]
Fetched 40,7 kB in 0s (250 kB/s)
Selecting previously unselected package tree.
(Reading database ... 342238 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for ma

In [7]:
# After downloading the data, we need to check the annotation format
!cat kinetics400_tiny/kinetics_tiny_train_video.txt

D32_1gwq35E.mp4 0
iRuyZSKhHRg.mp4 1
oXy-e_P_cAI.mp4 0
34XczvTaRiI.mp4 1
h2YqqUhnR34.mp4 0
O46YA8tI530.mp4 0
kFC3KY2bOP8.mp4 1
WWP5HZJsg-o.mp4 1
phDqGd0NKoo.mp4 1
yLC9CtWU5ws.mp4 0
27_CSXByd3s.mp4 1
IyfILH9lBRo.mp4 1
T_TMNGzVrDk.mp4 1
TkkZPZHbAKA.mp4 0
PnOe3GZRVX8.mp4 1
soEcZZsBmDs.mp4 1
FMlSTTpN3VY.mp4 1
WaS0qwP46Us.mp4 0
A-wiliK50Zw.mp4 1
oMrZaozOvdQ.mp4 1
ZQV4U2KQ370.mp4 0
DbX8mPslRXg.mp4 1
h10B9SVE-nk.mp4 1
P5M-hAts7MQ.mp4 0
R8HXQkdgKWA.mp4 0
D92m0HsHjcQ.mp4 0
RqnKtCEoEcA.mp4 0
LvcFDgCAXQs.mp4 0
xGY2dP0YUjA.mp4 0
Wh_YPQdH1Zg.mp4 0


According to the format defined in [`VideoDataset`](./datasets/video_dataset.py), each line indicates a sample video with the filepath and label, which are split with a whitespace.

### Modify the config

In the next step, we need to modify the config for the training.
To accelerate the process, we finetune a recognizer using a pre-trained recognizer.

In [1]:
#mmaction2 dir
%cd "/home/administrator/Z/Algorithms/mmaction2/"

/home/administrator/Z/Algorithms/mmaction2


In [8]:
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

Given a config that trains a TSN model on kinetics400-full dataset, we need to modify some values to use it for training TSN on Kinetics400-tiny dataset.


In [9]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'data/kinetics400_tiny/train/'
cfg.data_root_val = 'data/kinetics400_tiny/val/'
cfg.ann_file_train = 'data/kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = 'data/kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.ann_file_test = 'data/kinetics400_tiny/kinetics_tiny_val_video.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'data/kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.test.data_prefix = 'data/kinetics400_tiny/val/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'data/kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.data.train.data_prefix = 'data/kinetics400_tiny/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'data/kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.val.data_prefix = 'data/kinetics400_tiny/val/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 30

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNet',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False),
    cls_head=dict(
        type='TSNHead',
        num_classes=2,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.4,
        init_std=0.01),
    train_cfg=None,
    test_cfg=dict(average_clips=None))
optimizer = dict(type='SGD', lr=7.8125e-05, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=40, norm_type=2))
lr_config = dict(policy='step', step=[40, 80])
total_epochs = 30
checkpoint_config = dict(interval=10)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
resume_from = None
workflow = [('train', 1)]
dataset_type = 'VideoDataset'
data_root =

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [10]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

Use load_from_torchvision loader


2021-08-05 17:43:05,028 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.bias', 'fc.weight'}
2021-08-05 17:43:07,351 - mmaction - INFO - load checkpoint from ./checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
2021-08-05 17:43:07,353 - mmaction - INFO - Use load_from_local loader
2021-08-05 17:43:07,504 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint, the shape in current model is torch.Size([2, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([2]).
2021-08-05 17:43:07,505 - mmaction - INFO - Start running, host: administrator@administrator, work_dir: /home/administrator/Z/Algorithms/mmaction2/tutorial_exps
2021-08-05 17:43:07,505 - mmaction - INFO - Hooks will be executed in the following order:
b

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 10.1 task/s, elapsed: 1s, ETA:     0s

2021-08-05 17:43:30,233 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-08-05 17:43:30,235 - mmaction - INFO - 
top1_acc	0.8000
top5_acc	1.0000
2021-08-05 17:43:30,235 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-08-05 17:43:30,236 - mmaction - INFO - 
mean_acc	0.8000
2021-08-05 17:43:30,730 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_5.pth.
2021-08-05 17:43:30,731 - mmaction - INFO - Best top1_acc is 0.8000 at 5 epoch.
2021-08-05 17:43:30,732 - mmaction - INFO - Epoch(val) [5][5]	top1_acc: 0.8000, top5_acc: 1.0000, mean_class_accuracy: 0.8000
2021-08-05 17:43:33,861 - mmaction - INFO - Epoch [6][5/15]	lr: 7.813e-05, eta: 0:01:52, time: 0.625, data_time: 0.505, memory: 1620, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.5897, loss: 0.5897, grad_norm: 11.0825
2021-08-05 17:43:34,417 - mmaction - INFO - Epoch [6][10/15]	lr: 7.813e-05, eta: 0:01:47, time: 0.111, data_time: 0.001, memory: 1620, top1_acc: 0.6000, top5_acc: 1.0000, loss_cls

OSError: [Errno 95] Operation not supported: 'epoch_10.pth' -> '/home/administrator/Z/Algorithms/mmaction2/tutorial_exps/latest.pth'

### Understand the log
From the log, we can have a basic understanding the training process and know how well the recognizer is trained.

Firstly, the ResNet-50 backbone pre-trained on ImageNet is loaded, this is a common practice since training from scratch is more cost. The log shows that all the weights of the ResNet-50 backbone are loaded except the `fc.bias` and `fc.weight`.

Second, since the dataset we are using is small, we loaded a TSN model and finetune it for action recognition.
The original TSN is trained on original Kinetics-400 dataset which contains 400 classes but Kinetics-400 Tiny dataset only have 2 classes. Therefore, the last FC layer of the pre-trained TSN for classification has different weight shape and is not used.

Third, after training, the recognizer is evaluated by the default evaluation. The results show that the recognizer achieves 100% top1 accuracy and 100% top5 accuracy on the val dataset,
 
Not bad!

## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

In [40]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

# Build a test dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

eval_config = cfg.evaluation
eval_config.pop('interval')
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 6.0 task/s, elapsed: 2s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.8000
top5_acc	1.0000

Evaluating mean_class_accuracy ...

mean_acc	0.8000
top1_acc: 0.8000
top5_acc: 1.0000
mean_class_accuracy: 0.8000
